In [32]:
import pandas as pd
df = pd.read_csv('compustat.csv')
df.head()

/var/folders/kp/rt42wzwx7sz9plr9l175fkv40000gn/T/ipykernel_83096/313954475.py:2: DtypeWarning: Columns (10,12,13,17,18,20,25,26,30,32,33,34,35,647,648,649,650,654,656,664,668,669,674,675,678,679) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('compustat.csv')


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,2000-02-29,1999,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
1,1004,2000-05-31,1999,4.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
2,1004,2000-08-31,2000,1.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
3,1004,2000-11-30,2000,2.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
4,1004,2001-02-28,2000,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01


In [33]:
df_with_ratios = pd.DataFrame(df[['datadate', 'gvkey', 'fqtr', 'fyr', 'tic', 'sic']])
df_with_ratios.isna().sum()

datadate      0
gvkey         0
fqtr        346
fyr           0
tic         694
sic           0
dtype: int64

Looks like gvkey would be a better primary key than ticker since it has no nans in this dataset. We also need to inspect financial quarter

In [34]:
df_with_ratios[df['fqtr'].isna()]

,datadate,gvkey,fqtr,fyr,tic,sic
2746,2014-03-31,1224,NaN,12,EGN1,4924
2747,2014-06-30,1224,NaN,12,EGN1,4924
12300,2014-11-30,1951,NaN,8,PRSNQ,3845
31199,2014-12-31,3424,NaN,9,BH,5812
35114,2014-12-31,3806,NaN,9,DWSN,1382
...,...,...,...,...,...,...
1050126,2013-06-30,196766,NaN,3,ALGDF,1040
1050128,2013-09-30,196766,NaN,3,ALGDF,1040
1074260,2015-06-30,317427,NaN,3,NOMD,2030
1074262,2015-09-30,317427,NaN,3,NOMD,2030


No clear pattern in financial quarters with nan values. Just use date value since there is no nans there

In [35]:
df_with_ratios.drop(['fqtr', 'fyr'], axis=1, inplace=True)
df_with_ratios.head()

,datadate,gvkey,tic,sic
0,2000-02-29,1004,AIR,5080
1,2000-05-31,1004,AIR,5080
2,2000-08-31,1004,AIR,5080
3,2000-11-30,1004,AIR,5080
4,2001-02-28,1004,AIR,5080


In [36]:
df['average_asset'] = (df['atq'] + df['atq'].shift(1)) / 2
df['average_inventory'] = (df['invtq'] + df['invtq'].shift(1)) / 2
df['average_receivables'] = (df['rectq'] + df['rectq'].shift(1)) / 2
df['average_workingcapital'] = (df['wcapq'] + df['wcapq'].shift(1)) / 2
df['cogs'] = df['cogsq'] / (1e-6 + df['cogsy'])

In [37]:
df_with_ratios['current_ratio'] = df['actq'] / (1e-6 + df['lctq'])
df_with_ratios['quick_ratio'] = (df['actq'] - df['invtq']) / (1e-6 + df['lctq']) 
df_with_ratios['cash_ratio'] = df['chechy'] / (1e-6 + df['lctq'] ) 
df_with_ratios['net_working_capital'] = df['actq'] - df['lctq']
df_with_ratios['debt_ratio'] = (df['dlttq'] + df['dlcq']) / (1e-6 + df['atq'])
df_with_ratios['debt_to_equity_ratio'] = (df['dlttq'] + df['dlcq']) / (1e-6 + df['ceqq']) 
df_with_ratios['equity_ratio'] = df['ceqq'] / (1e-6 + df['atq'])
df_with_ratios['cashflow_to_debt_ratio'] = df['oancfy'] / (1e-6 + (df['dlttq'] + df['dlcq']))
df_with_ratios['net_profit_margin'] = 100 * (df['niq'] / (1e-6 + df['revtq']))
df_with_ratios['return_on_assets'] = 100 * (df['niq'] / (1e-6 + df['atq']))
df_with_ratios['asset_turnover'] = df['revtq'] / (1e-6 + df['average_asset'])
df_with_ratios['inventory_turnover'] = df['cogs'] / (1e-6 + df['average_inventory'])
df_with_ratios['days_in_inventory'] = 365 / (1e-6 + (df['cogs'] / (1e-6 + df['average_inventory'])))
df_with_ratios['receivables_turnover'] = df['revtq'] / (1e-6 + df['average_inventory'])
df_with_ratios['day_sales_outstanding'] = 365 / (1e-6 + (df['revtq'] / (1e-6 + df['average_inventory'])))
df_with_ratios['working_capital_turnover'] = df['revtq'] / (1e-6 + df['average_workingcapital'])
df_with_ratios['price_to_earnings'] = df['prccq'] / (1e-6 + df['epsfxq'])
df_with_ratios['dividend_payout_ratio'] = df['dvpspq'] / (1e-6 + df['epsfxq'])
df_with_ratios['retention_ratio'] = 1 - (df['dvpspq'] / (1e-6 + df['epsfxq']))
df_with_ratios['gross_margin_ratio'] = 100 * ((df['revtq'] - df['cogsq']) / (1e-6 + df['revtq']))
df_with_ratios['operating_profit_margin'] = 100 * ((df['revtq'] - df['cogsq']- df['xoprq']) / (1e-6 + df['revtq']))
df_with_ratios['ebitda_margin'] = 100 * ((df['revtq'] - df['cogsq']- df['xoprq'] + df['dpq']) / (1e-6 + df['revtq']))
df_with_ratios['debt_service_coverage_ratio'] = (df['revtq'] - df['cogsq']- df['xoprq']) / (1e-6 + (df['dlttq'] + df['dlcq']))
df_with_ratios['interest_coverage_ratio'] = df['oibdpq'] / (1e-6 + df['xintq'])

In [38]:
df_with_ratios.head(5)

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,day_sales_outstanding,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,NaN,NaN,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,531.051575,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,513.171702,0.701345,93.750000,0.708333,0.291667,16.176118,-77.799562,-75.858047,-0.823112,2.432365
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,594.639896,0.633329,64.843750,0.531250,0.468750,18.857738,-73.583647,-71.446755,-0.674333,2.793634
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,637.845577,0.584946,68.000000,0.425000,0.575000,20.259808,-71.119752,-68.726602,-0.619269,3.174489


In [39]:
print(f"length of dataset: {len(df_with_ratios)}")
print(f"shape of dataset: {df_with_ratios.shape}")

length of dataset: 1075571
shape of dataset: (1075571, 28)


In [40]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.064524
sic,0.000000
current_ratio,37.248122
quick_ratio,38.371990
cash_ratio,38.470171
net_working_capital,37.207679
debt_ratio,28.442846
debt_to_equity_ratio,28.606852


At this point our dataset should have columns with at most 20-30% nan values. we shall now inpute them using sector median. 
Let us first try Methodology 1: 
1. map gvkey to sector
2. Eg. if company X in sector A on date 2000-01-01 has a missing value for total income, we inpute it using the median of sector A in 2000-01-01. 

This assumes that on a particular date, a companies fundamentals can be represented by its sector median on that date. 
Sector will be represented by SIC code. (I used all 4 digits for now)

First let us convert datadate to year and quarter representation

In [41]:
df_with_ratios['datadate'] = pd.to_datetime(df_with_ratios['datadate'])
df_with_ratios['year'] = df_with_ratios['datadate'].dt.year
df_with_ratios['quarter'] = df_with_ratios['datadate'].dt.quarter
df_with_ratios.head()

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,2000,1
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,2000,2
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,93.750000,0.708333,0.291667,16.176118,-77.799562,-75.858047,-0.823112,2.432365,2000,3
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,64.843750,0.531250,0.468750,18.857738,-73.583647,-71.446755,-0.674333,2.793634,2000,4
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,68.000000,0.425000,0.575000,20.259808,-71.119752,-68.726602,-0.619269,3.174489,2001,1


In [42]:
df_drop_duplicates = df_with_ratios.drop_duplicates(subset=['gvkey', 'year', 'quarter'])
len(df_with_ratios) - len(df_drop_duplicates)

1402

We then drop rows where there are duplicate year, quarters and gvkey, since the original dataset is monthly data. We only drop 1402 rows, so not significant. 

In [43]:
df_with_ratios = df_drop_duplicates

Now we find all the gvkeys where there is no data at all and drop them from our dataframe. From this operation we remove 6813 gvkeys from a total of 

In [44]:
df_drop_gvkey = df_with_ratios.drop(['sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
df_drop_gvkey = df_drop_gvkey.groupby('gvkey').apply(lambda group: group.iloc[:, 1:].isna().all().all())
to_drop = df_drop_gvkey[df_drop_gvkey == True]
gvkeys_to_drop = list(to_drop.index)

In [45]:
original_number_of_gvkeys = len(df_with_ratios['gvkey'].unique())
print(f"Dropped {len(gvkeys_to_drop) * 100 / original_number_of_gvkeys}% of stocks")

Dropped 22.53381394887397% of stocks


In [46]:
df_with_ratios_dropped_gvkey = df_with_ratios[~df_with_ratios['gvkey'].isin(gvkeys_to_drop)]
print(f"number of unique gvkeys now: {len(df_with_ratios_dropped_gvkey['gvkey'].unique())}")
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey)) * 100 / len(df_with_ratios)  }%")

number of unique gvkeys now: 23425
% of total rows lost: 17.750651899282143%


In [47]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey.isna().sum() / len(df_with_ratios_dropped_gvkey)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.027957
sic,0.000000
current_ratio,23.707834
quick_ratio,25.075580
cash_ratio,25.167941
net_working_capital,23.658937
debt_ratio,12.990197
debt_to_equity_ratio,13.190650


We can also see that the % of nans has decreased significantly across columns after dropping these gvkeys

In [48]:
test = df_with_ratios_dropped_gvkey
test = test[test.isna().any(axis=1)]
test[test['tic'] == 'AFAP']

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
156,2000-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,84.745763,0.282486,0.717514,NaN,NaN,NaN,NaN,NaN,2000,1
157,2000-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,49.848943,0.151057,0.848943,NaN,NaN,NaN,NaN,NaN,2000,2
158,2000-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,3
159,2000-12-31,1019,AFAP,7380,1.640074,1.550835,0.116883,3.450,0.062365,0.135467,...,34.014870,0.092937,0.907063,-17.483221,-195.148610,-185.426654,-22.792833,24.408072,2000,4
160,2001-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,75.100402,0.200803,0.799197,NaN,NaN,NaN,NaN,NaN,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-12-31,1019,AFAP,7380,1.405545,1.159588,0.031515,6.846,0.054954,0.157569,...,13.225569,0.036738,0.963262,32.806181,-60.904102,-58.519883,-27.607201,44.666667,2019,4
236,2020-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1
237,2020-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2
238,2020-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,3


Here we can see that there are still certain stocks where there is data, but many missing values. for example for ticker AFAP, we have data missing from 2003-2016. Such tickers should be dropped as well

In [49]:
cols_df = df_with_ratios_dropped_gvkey.drop(['gvkey', 'sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
all_na_rows = cols_df[cols_df.isna().all(axis=1)]
df_with_nans = df_with_ratios_dropped_gvkey.loc[all_na_rows.index]
df_with_nans_counts = df_with_nans.groupby('gvkey').size()
gvkey_with_bad_data = df_with_nans_counts[df_with_nans_counts > 10]

In [50]:
print(len(df_with_ratios_dropped_gvkey['gvkey'].unique()))
print(100*len(gvkey_with_bad_data) / len(df_with_ratios_dropped_gvkey['gvkey'].unique()))

23425
2.992529348986126


We can decide here the threshold for which we want to drop our gvkeys. How many nan rows can we allow a gvkey to have? For now i set the value at less than 10. This causes us to drop 671 gvkeys, or 2.86% of gvkeys. 

In [51]:
df_with_ratios_dropped_gvkey_v2 = df_with_ratios_dropped_gvkey[~df_with_ratios_dropped_gvkey['gvkey'].isin(gvkey_with_bad_data.index)]
len(df_with_ratios_dropped_gvkey_v2)

843144

In [52]:
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey_v2)) * 100 / len(df_with_ratios)  }%")

% of total rows lost: 21.50732333552728%


In [53]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.022995
sic,0.000000
current_ratio,17.390932
quick_ratio,18.483870
cash_ratio,18.525577
net_working_capital,17.353787
debt_ratio,8.907723
debt_to_equity_ratio,9.064216


Our % of rows dropped increased from 17% to 21%. We can reduce this perentage by increasing the tolerance of how many nan rows a stock can have. This is balancing data quantity vs quality. 

In [54]:
# dealing with inf value
inf_breakdown = pd.DataFrame()
inf_breakdown['pct_inf'] = (100 * df_with_ratios_dropped_gvkey_v2.isin([float('inf'), float('-inf')]).sum()) / len(df_with_ratios_dropped_gvkey_v2)
inf_breakdown

,pct_inf
datadate,0.000000
gvkey,0.000000
tic,0.000000
sic,0.000000
current_ratio,0.082430
quick_ratio,0.082311
cash_ratio,0.071874
net_working_capital,0.000000
debt_ratio,0.182531
debt_to_equity_ratio,0.011742


In [135]:
df_with_ratios_dropped_gvkey_v2['cashflow_to_debt_ratio'].quantile(0.1)

In [137]:
sector_date_median = df_with_ratios_dropped_gvkey_v2.groupby(['sic', 'year', 'quarter']).median()
sector_date_median

,pct_inf
current_ratio,0.0
quick_ratio,0.0
cash_ratio,0.0
net_working_capital,0.0
debt_ratio,0.0
debt_to_equity_ratio,0.0
equity_ratio,0.0
cashflow_to_debt_ratio,0.0
net_profit_margin,0.0
return_on_assets,0.0


In [140]:
cols_to_check = [col for col in df_with_ratios_dropped_gvkey_v2.columns if col not in ['datadate', 'year', 'quarter', 'sic', 'gvkey', 'tic']]
cols_to_check

In [142]:
import numpy as np
def fill_with_median(x):
    sic = x['sic']
    year = x['year']
    quarter = x['quarter']
    for col in cols_to_check: 
        if np.isnan(x[col]): 
            median = sector_date_median[sector_date_median.index == (sic,year,quarter)][col].iloc[0]
            x[col] = median
    return x

,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,equity_ratio,cashflow_to_debt_ratio,net_profit_margin,return_on_assets,...,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,gvkey,sic,tic,year,quarter,datadate
0,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,0.454368,0.027511,4.022678,1.453390,...,-72.491564,-70.807951,-0.958372,4.223950,1004,5080,AIR,2000,1,2000-02-29
1,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,0.458186,0.048612,1.097837,0.333469,...,-75.280235,-73.157425,-0.819497,2.287927,1004,5080,AIR,2000,2,2000-05-31
2,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,0.453824,-0.033884,1.306614,0.422584,...,-77.799562,-75.858047,-0.823112,2.432365,1004,5080,AIR,2000,3,2000-08-31
3,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,0.441514,0.044014,2.024274,0.553470,...,-73.583647,-71.446755,-0.674333,2.793634,1004,5080,AIR,2000,4,2000-11-30
4,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,0.456946,0.008334,2.693044,0.713909,...,-71.119752,-68.726602,-0.619269,3.174489,1004,5080,AIR,2001,1,2001-02-28


In [144]:
df_inputed1 = df_with_ratios_dropped_gvkey_v2[:50000].apply(fill_with_median, axis=1)
df_inputed2 = df_with_ratios_dropped_gvkey_v2[50000:100000].apply(fill_with_median, axis=1)
df_inputed3 = df_with_ratios_dropped_gvkey_v2[100000:150000].apply(fill_with_median, axis=1)

current_ratio  quick_ratio  cash_ratio  \
sic  year quarter                                           
100  2000 1             1.647509     0.737603   -0.005996   
          2             1.814017     0.896253   -0.006626   
          3             1.507615     0.904313   -0.004942   
          4             1.577333     0.855120    0.000416   
     2001 1             1.650396     0.930933    0.001662   
...                          ...          ...         ...   
9998 2010 4            13.658537    13.658537   -8.469512   
     2011 1            15.833837    15.833837    2.410876   
          2            15.663768    15.663768    2.646377   
          3            17.950156    17.950156    3.532710   
          4            20.237931    20.237931    4.906897   

                   net_working_capital  debt_ratio  debt_to_equity_ratio  \
sic  year quarter                                                          
100  2000 1                    20.4455    0.420928              0.854158   
          2                    22.4370    0.390419              0.736070   
          3                    21.1725    0.385024              0.819076   
          4                    19.9730    0.378760              0.842804   
     2001 1                    17.5215    0.376966              0.800000   
...                                ...         ...                   ...   
9998 2010 4                     4.1520    0.000000              0.000000   
     2011 1                     4.9100    0.000000              0.000000   
          2                     5.0590    0.000000              0.000000   
          3                     5.4410    0.000000              0.000000   
          4                     5.5790    0.000000              0.000000   

                   equity_ratio  cashflow_to_debt_ratio  net_profit_margin  \
sic  year quarter                                                            
100  2000 1            0.424581               -0.046012       2.629500e+00   
          2            0.434415                0.054531       2.120489e+00   
          3            0.446250                0.046320      -7.016296e+00   
          4            0.467537                0.014073      -4.486626e+00   
     2001 1            0.474421                0.002461       3.246027e+00   
...                         ...                     ...                ...   
9998 2010 4            0.986136           -32257.000000       4.395000e+04   
     2011 1            0.985965           -32257.000000      -2.901190e+07   
          2            0.986099           -32257.000000       4.395000e+04   
          3            0.987607           -32257.000000       4.395000e+04   
          4            0.988962           -32257.000000       4.395000e+04   

                   return_on_assets  ...  working_capital_turnover  \
sic  year quarter                    ...                             
100  2000 1                0.387070  ...                  0.269688   
          2                0.306799  ...                  0.645276   
          3               -0.895477  ...                  0.577829   
          4               -0.695809  ...                  1.754690   
     2001 1                0.459175  ...                  0.898301   
...                             ...  ...                       ...   
9998 2010 4                1.327247  ...                  0.000000   
     2011 1               -0.326493  ...                  0.000000   
          2                4.919618  ...                  0.000000   
          3                4.270105  ...                  0.000000   
          4                1.533894  ...                  0.000000   

                   price_to_earnings  dividend_payout_ratio  retention_ratio  \
sic  year quarter                                                              
100  2000 1                11.902778               0.000000         1.000000   
          2                 0.122622               0.000000         1.

In [145]:
df_inputed4 = df_with_ratios_dropped_gvkey_v2[150000:200000].apply(fill_with_median, axis=1)
df_inputed5 = df_with_ratios_dropped_gvkey_v2[200000:250000].apply(fill_with_median, axis=1)
df_inputed6 = df_with_ratios_dropped_gvkey_v2[250000:300000].apply(fill_with_median, axis=1)

['current_ratio',
 'quick_ratio',
 'cash_ratio',
 'net_working_capital',
 'debt_ratio',
 'debt_to_equity_ratio',
 'equity_ratio',
 'cashflow_to_debt_ratio',
 'net_profit_margin',
 'return_on_assets',
 'asset_turnover',
 'inventory_turnover',
 'days_in_inventory',
 'receivables_turnover',
 'day_sales_outstanding',
 'working_capital_turnover',
 'price_to_earnings',
 'dividend_payout_ratio',
 'retention_ratio',
 'gross_margin_ratio',
 'operating_profit_margin',
 'ebitda_margin',
 'debt_service_coverage_ratio',
 'interest_coverage_ratio']

In [146]:
df_inputed7 = df_with_ratios_dropped_gvkey_v2[300000:350000].apply(fill_with_median, axis=1)
df_inputed8 = df_with_ratios_dropped_gvkey_v2[350000:400000].apply(fill_with_median, axis=1)
df_inputed9 = df_with_ratios_dropped_gvkey_v2[400000:450000].apply(fill_with_median, axis=1)

In [147]:
combined = pd.DataFrame()
for df in [
    df_inputed1,
    df_inputed2,
        df_inputed3,
    df_inputed4,
        df_inputed5,
    df_inputed6,
        df_inputed7,
    df_inputed8,
    df_inputed9,
]:
    combined = pd.concat([combined, df])
    
    

In [148]:
df_inputed10 = df_with_ratios_dropped_gvkey_v2[450000:500000].apply(fill_with_median, axis=1)
df_inputed11 = df_with_ratios_dropped_gvkey_v2[500000:550000].apply(fill_with_median, axis=1)
df_inputed12 = df_with_ratios_dropped_gvkey_v2[550000:600000].apply(fill_with_median, axis=1)

In [149]:
df_inputed13 = df_with_ratios_dropped_gvkey_v2[600000:650000].apply(fill_with_median, axis=1)
df_inputed14 = df_with_ratios_dropped_gvkey_v2[650000:700000].apply(fill_with_median, axis=1)
df_inputed15 = df_with_ratios_dropped_gvkey_v2[700000:750000].apply(fill_with_median, axis=1)

In [150]:

df_inputed16 = df_with_ratios_dropped_gvkey_v2[750000:800000].apply(fill_with_median, axis=1)
df_inputed17 = df_with_ratios_dropped_gvkey_v2[800000:850000].apply(fill_with_median, axis=1)

In [151]:
for df in [
    df_inputed10,
    df_inputed11,
        df_inputed12,
    df_inputed13,
        df_inputed14,
    df_inputed15,
        df_inputed16,
    df_inputed17,
]:
    combined = pd.concat([combined, df])

In [152]:
combined.to_pickle('./combined_final_22_oct.pkl')

In [153]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (combined.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

In [154]:
inputed_data_ffill = combined.ffill(limit=4)

In [155]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (combined.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute_and_ffill'] = (inputed_data_ffill.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

In [156]:
print(len(df_with_ratios))
print(len(inputed_data_ffill))

843144

In [165]:
inputed_data_ffill_dropna = inputed_data_ffill.dropna()

In [171]:
len(inputed_data_ffill_dropna)

(843144, 30)

In [167]:
print(len(df_with_ratios['gvkey'].unique()))
print(len(inputed_data_ffill_dropna['gvkey'].unique()))

In [168]:
inputed_data_ffill_dropna.to_pickle('./compustat_inputed.pkl')

,pct_nan_before_sector_inpute,pct_nan_after_sector_inpute,pct_nan_after_sector_inpute_and_ffill
datadate,0.000000,0.000000,0.000000
gvkey,0.000000,0.000000,0.000000
tic,0.022995,0.022995,0.020388
sic,0.000000,0.000000,0.000000
current_ratio,17.390932,6.011438,4.738361
quick_ratio,18.483870,6.048303,4.740316
cash_ratio,18.525577,6.116356,4.601976
net_working_capital,17.353787,6.000173,4.738267
debt_ratio,8.907723,0.037145,0.001490
debt_to_equity_ratio,9.064216,0.038076,0.001490


In [169]:
inputed_data_ffill_dropna

1074169
843144
